In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout #TimeoutError already exists in python so rename
import time

In [2]:
SEASONS = list(range(2016,2025)) #range will go from 2016-2024
SEASONS

[2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

In [3]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [4]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep*i) #sleep more on each retry to avoid ban
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch() #wait for new browser to launch 'await'
                page = await browser.new_page() #create new tab
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector) #pass selector to grab only piece of html
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue #go to top of loop and retry
        else:
            break #successful scrape, so stop loop
    return html

In [5]:
season = 2016

url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"

In [6]:
url

'https://www.basketball-reference.com/leagues/NBA_2016_games.html'

In [7]:
#testing html on single season
html = await get_html(url, "#content .filter") #we pass in selector, find div with id content, then with class filter 

2015-16 NBA Schedule | Basketball-Reference.com


In [6]:
#Scrape directly from BBRef the month html pages for a season
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a") # find all the a tag links
    #now we place the basketball-ref url in front of the "/leagues/NBA_{year}_games-{month}.html"
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links] 
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])  #save in standings dir, and make filename the ending that has year/month
        if os.path.exists(save_path):
            print(f"Skipping {save_path} since it is already saved. If you intended to overwrite, delete file and run again!") 
            continue # don't double save
        print(f"Saving {save_path}...")
        #Inside a specific month for that season, grab entire table for box scores and save in path
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [12]:
#Scrape all the seasons (not needed anymore since we already scraped 2016-2024)
#for season in SEASONS:
    #await scrape_season(season)

In [7]:
#Scrape just a single season (2025 is updated as of 1/29/24, 2024 and below doesn't need to be updated)
#NOTE: If you want to update the 2025 pages so they're more current, need to delete old 2025_games-month files b/c skips it
await scrape_season(2025)

2024-25 NBA Schedule | Basketball-Reference.com
Skipping data/standings/NBA_2025_games-october.html since it is already saved. If you intended to overwrite, delete file and run again!
Skipping data/standings/NBA_2025_games-november.html since it is already saved. If you intended to overwrite, delete file and run again!
Skipping data/standings/NBA_2025_games-december.html since it is already saved. If you intended to overwrite, delete file and run again!
Skipping data/standings/NBA_2025_games-january.html since it is already saved. If you intended to overwrite, delete file and run again!
Saving data/standings/NBA_2025_games-february.html...
2024-25 NBA Schedule | Basketball-Reference.com
Skipping data/standings/NBA_2025_games-march.html since it is already saved. If you intended to overwrite, delete file and run again!
Skipping data/standings/NBA_2025_games-april.html since it is already saved. If you intended to overwrite, delete file and run again!


In [8]:
#Check current standing files (months with yet to be played games will have no box score link, need to be re-donee later)
standings_files = os.listdir(STANDINGS_DIR)
standings_files = [s for s in standings_files if ".html" in s] #ignore non-html files like auto generated jupyter files
standings_files

['NBA_2016_games-april.html',
 'NBA_2016_games-december.html',
 'NBA_2016_games-february.html',
 'NBA_2016_games-january.html',
 'NBA_2016_games-june.html',
 'NBA_2016_games-march.html',
 'NBA_2016_games-may.html',
 'NBA_2016_games-november.html',
 'NBA_2016_games-october.html',
 'NBA_2017_games-april.html',
 'NBA_2017_games-december.html',
 'NBA_2017_games-february.html',
 'NBA_2017_games-january.html',
 'NBA_2017_games-june.html',
 'NBA_2017_games-march.html',
 'NBA_2017_games-may.html',
 'NBA_2017_games-november.html',
 'NBA_2017_games-october.html',
 'NBA_2018_games-april.html',
 'NBA_2018_games-december.html',
 'NBA_2018_games-february.html',
 'NBA_2018_games-january.html',
 'NBA_2018_games-june.html',
 'NBA_2018_games-march.html',
 'NBA_2018_games-may.html',
 'NBA_2018_games-november.html',
 'NBA_2018_games-october.html',
 'NBA_2019_games-april.html',
 'NBA_2019_games-december.html',
 'NBA_2019_games-february.html',
 'NBA_2019_games-january.html',
 'NBA_2019_games-june.html',
 'N

In [9]:
SUCCESS = 0
NO_BOX_SCORES = 1
NO_NEW_GAMES = 2
#Scrape a standings file for a single month/season and save box score in scores dir
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()
    soup = BeautifulSoup(html)
    links = soup.find_all("a") #again, get all the a tag elements, but we'll have to get only box score links in the table for each game
    hrefs = [l.get("href") for l in links]  #get only the href elements in an array
    box_scores_for_month = [l for l in hrefs if l is not None and "boxscore" in l and ".html" in l] #filter only html box scores
    if not box_scores_for_month:
        return NO_BOX_SCORES
    box_scores_for_month_url = [f"https://www.basketball-reference.com{l}" for l in box_scores_for_month] # loop through each and append start of link
    has_new_save = False
    for url in box_scores_for_month_url:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1]) #get last part of url as file name
        #Skip any already saved box scores to make sure we don't wait and risk rate limit for duplicates
        if os.path.exists(save_path):
            continue
        has_new_save = True # Flip to true since new game found
        html = await get_html(url, "#content") #select id content (will contain scorebox)
        if not html: #if we retried 3 times and still don't have html, just continue loop to next box score
            continue
        with open(save_path, "w+") as f: #else, save it
            f.write(html)
    if not has_new_save:
        return NO_NEW_GAMES
    else:
        return SUCCESS

In [10]:
#Run this again to finish parsing, itll skip already saved games because of the "if os.path.exists" line above
#UPDATE: Up to date as of 2/02/25, need to re-scrape using "await scrape_season(2025)" above for new box score links 4 2025, then run this
for s in standings_files:
    standings_file_path = os.path.join(STANDINGS_DIR, s) #s will only have the NBA_year_games-month.html, so give it full path 
    retCode = await scrape_game(standings_file_path)
    if retCode == NO_BOX_SCORES:
        print(f"{s} had no box score links and is likely a future month in season.")
    elif retCode == NO_NEW_GAMES:
        print(f"Every game in {s} has already been saved.")
print("Done!")

Every game in NBA_2016_games-april.html has already been saved.
Every game in NBA_2016_games-december.html has already been saved.
Every game in NBA_2016_games-february.html has already been saved.
Every game in NBA_2016_games-january.html has already been saved.
Every game in NBA_2016_games-june.html has already been saved.
Every game in NBA_2016_games-march.html has already been saved.
Every game in NBA_2016_games-may.html has already been saved.
Every game in NBA_2016_games-november.html has already been saved.
Every game in NBA_2016_games-october.html has already been saved.
Every game in NBA_2017_games-april.html has already been saved.
Every game in NBA_2017_games-december.html has already been saved.
Every game in NBA_2017_games-february.html has already been saved.
Every game in NBA_2017_games-january.html has already been saved.
Every game in NBA_2017_games-june.html has already been saved.
Every game in NBA_2017_games-march.html has already been saved.
Every game in NBA_2017_